In [ ]:
import tensorflow.keras as kr
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
def path_to_array(path, img_size):
    
    """
    returns a (1, img_size, img_size, 3) matrix from values from 0 to 1
    """
    
    img_path = path
    img = kr.preprocessing.image.load_img(img_path, target_size=(img_size, img_size))
    x = kr.preprocessing.image.img_to_array(img)
    x /= 255
    x = tf.expand_dims(x, 0)
    return x

In [ ]:
def show_image(img):
    """
    img is a array of shape length, bredth, 3 and elements of img are in range of 0-1

    """
    
    plt.figure()
    plt.imshow(img)

In [ ]:
def predict(img, model):
    
    """
    
    img is a array same as in show_img
    model is a model
        
    """

    return model.predict(img)

In [ ]:
def intermediate_activation(img, model, layer, conv_net):
    l = model.get_layer(index = layer)
    layer_outputs = l.output[0, :, :, conv_net]
    activation_model = kr.models.Model(inputs = model.input, outputs = layer_outputs)
    vis = activation_model.predict(img)
    return vis

In [ ]:
def feature_activation(model, layer, filter_no, step = 1, epoch=40):

    input_img_data = np.random.random((1, model.layers[0].input.shape[1], model.layers[0].input.shape[2], 3)) * 20 + 128
    input_img_data = tf.Variable(input_img_data, dtype="float32")
    acti_model = kr.models.Model(inputs = model.input, outputs = model.get_layer(index = layer).output[:, :, :, filter_no])
    for i in range(epoch):
        with tf.GradientTape(persistent=True) as gt:
            output= acti_model(input_img_data)
            loss = kr.backend.mean(output)
        
        gradients = gt.gradient(loss, input_img_data)        
        gradients /= (kr.backend.sqrt(kr.backend.mean(kr.backend.square(gradients))) + 1e-5)
        
        input_img_data.assign_add(gradients * step)
        
    input_img_data = input_img_data.numpy()    
    input_img_data -= input_img_data.mean()
    input_img_data /= (input_img_data.std() + 1e-5)
    input_img_data *= 0.1
    input_img_data += 0.5
    input_img_data = np.clip(input_img_data, 0, 1)
    input_img_data *= 255
    input_img_data = np.clip(input_img_data, 0, 255).astype('uint8')

    return input_img_data[0]

In [ ]:
def heat_map(img, model, last_conv_layer = -4):
    
    """
    img is 1, length, bredth, 3
    
    output is 1, length, bredth, 3
    """

    acti_model = kr.models.Model(inputs = model.input, outputs = [model.layers[last_conv_layer].output, model.output])       
        
    with tf.GradientTape() as gt:
         
        int_output, fin_output= acti_model(img)
        grads = gt.gradient(fin_output, int_output)

    pooled_grads = kr.backend.mean(grads, axis=(0, 1, 2))
    
    for i in range(int_output.shape[3]):
        output = int_output.numpy()
        output[:, :, :, i] *= pooled_grads[i]
    
    heatmap = np.mean(int_output, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    return heatmap

In [ ]:
def superimpose(main_img, secondary_img):
    secondary_img = secondary_img.reshape((secondary_img.shape[1], secondary_img.shape[2], 1))
    secondary_img = tf.image.resize(secondary_img, (main_img.shape[1], main_img.shape[2]))
    secondary_img = secondary_img.numpy().reshape((1, secondary_img.shape[0], secondary_img.shape[1]))
    
    main_img = tf.image.rgb_to_grayscale(tf.constant(main_img))
    main_img = main_img.numpy()[:, :, :, 0]

    superimposed_image = (main_img + secondary_img * 0) / 255
    return superimposed_image